<a href="https://colab.research.google.com/github/andres-merino/Generador-Moodle-LaTeX/blob/main/Codigo%20Colab%20v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generador de preguntas de Moodle

In [4]:
# Paquetes
import itertools
import os
import random

## Definiciones

Llenar estos datos, en el enunciado, los parámetros y datos que deseen sean reemplazados o calculados por el generador, deben estar entre corchetes dobles `[[ ]]`. Dentro de cada pregunta, si un item no tiene opciones, dejar corchetes vacíos `[]` (ver ejemplo). El identificador de la pregunta siempre debe tener el parámetro `[[id]]` para que sea reemplazado por el generador.

In [5]:
# Nombre del cuestionario
name = "Producto"

# Parámetros
par = ["a", "b"]
# Diccionario de parámetros
par_dict = {
    "a": [-2,-1,0,1,2],
    "b": [-2,-1,1,2],
    }

# Enunciado
enunciado = r'''
\begin{numerical}[]%
    % - Indentificador
    {Producto - [[id]]}
    % - Enunciado
    Determine el valor de
    \[
        ([[a]]) \cdot ([[b]])
    \]
    \item[] [[a*b]]
\end{numerical}
'''

# Cantidad de preguntas a generar
n_preguntas = 2

# Código adicional a compilar antes de la generación de cada pregunta (opcional)
# funciones = []
# Plantilla personalizada (opcional, por defento Plantilla.tex)
# plantilla = "PlantillaAleph.tex"
# Carpeta de salida (opcional, por defecto output/)
# carpeta = "output/"

## Probar con un parámetro

Para probar con un parámetro, ejecutar el siguiente código. Si se quiere solo generar una cantidad $n$ de preguntas para copiarlas, cambiar el valor de total preguntas.


In [6]:
# Total preguntas
_total = n_preguntas
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(18)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
_par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    # try:
    for _n, _parametro in enumerate(par):
        exec(f"{_parametro} = _parametros[{_n}]")
    for _funcion in funciones:
        exec(_funcion)
    # Reemplazo
    _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
    exec(f'_quiz += fr"""{_enunciado_F}"""')
    _quiz += '\n'
    id += 1
    # except:
        # print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
            break

# Mensaje
print(_quiz)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[]%
    % - Indentificador
    {Producto - 1}
    % - Enunciado
    Determine el valor de
    \[
        (-1) \cdot (-1)
    \]
    \item[] 1
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[]%
    % - Indentificador
    {Producto - 2}
    % - Enunciado
    Determine el valor de
    \[
        (-2) \cdot (2)
    \]
    \item[] -4
\end{numerical}




## Generar el examen

Si todo está bien, ejecutar el siguiente código para generar el examen.

In [7]:
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))

# Semilla
random.seed(18)

# Si está definido el máximo de ejercicios
if n_preguntas:
    # Tomo combinaciones aleatorias
    _par_comb = random.sample(_par_comb, n_preguntas)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        for _funcion in funciones:
            exec(_funcion)
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)

_parametros_latex = ''
for _parametro in par:
    _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\\{' + ', '.join(map(str, par_dict[_parametro])) + '\\}$' + '\n'

# Leo el archivo plantilla
# Si la variable plantilla no está definida, la defino como 'Plantilla.tex'
if not 'plantilla' in locals():
    plantilla = 'Plantilla.tex'
_plantilla = open(plantilla, 'r', encoding='utf-8')
_t_plantilla = _plantilla.read()
_plantilla.close()
# Reemplazo
_t_plantilla = _t_plantilla.replace('{{QUIZ}}', _quiz)
_t_plantilla = _t_plantilla.replace('{{Cuestionario}}', name)
_t_plantilla = _t_plantilla.replace('{{Parámetros}}', _parametros_latex)
_t_plantilla = _t_plantilla.replace('{{Número de preguntas}}', str(id-1))

# Escribo el archivo
# Si la variable carpeta no está definida, la defino como 'test'
if not 'carpeta' in locals():
    carpeta = 'output'
# Si no existe la carpeta, la creo
if not os.path.exists(carpeta):
    os.makedirs(carpeta)
_examen = open(f'{carpeta}/{name}.tex', 'w', encoding='utf-8')
_examen.write(_t_plantilla)
_examen.close()

# Mensaje
print(f'Examen generado con éxito, se generaron {id-1} preguntas.')

Examen generado con éxito, se generaron 2 preguntas.


<>:45: SyntaxWarning: invalid escape sequence '\{'
<>:45: SyntaxWarning: invalid escape sequence '\}'
<>:45: SyntaxWarning: invalid escape sequence '\{'
<>:45: SyntaxWarning: invalid escape sequence '\}'
C:\Users\aemerinot\AppData\Local\Temp\ipykernel_18552\2270743655.py:45: SyntaxWarning: invalid escape sequence '\{'
  _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\{' + ', '.join(map(str, par_dict[_parametro])) + '\}$' + '\n'
C:\Users\aemerinot\AppData\Local\Temp\ipykernel_18552\2270743655.py:45: SyntaxWarning: invalid escape sequence '\}'
  _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\{' + ', '.join(map(str, par_dict[_parametro])) + '\}$' + '\n'
